# 机器学习测试

In [2]:
# 数据接口 
import akshare as ak
import baostock as bs
import tushare as ts

# 基础模块
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import time

# 机器学习模块
import xgboost as xgb

# 回测框架
import backtrader as bt
import backtrader.indicators as btind

# 基础函数
import utilsJ

### 策略

In [62]:
class ml1(bt.Strategy):
    
    params = (
        # General params
        ('printlog', False),
        ('units', 1),
        ('p_stake', 100),
    )
    
    
    def log(self, txt, dt=None, doprint=False):
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s: %s' % (dt.isoformat(), txt))
            #with open('log.txt', 'a') as file:
                #file.write('%s: %s \n' % (dt.isoformat(), txt))
        
    
    def __init__(self):
        # Keep references to lines in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.datahigh = self.datas[0].high
        self.datalow = self.datas[0].low

        # Keep references to executed order & price
        self.buyprice = 0
        self.sellprice = 0
        self.order = None

        # Add indicators

        self.bollinger = btind.BollingerBands(period=20, devfactor=2)
        self.macdhisto = btind.MACDHisto()
        self.wr = btind.WilliamsR(period=14)

        self.updown = bt.If(self.dataclose(0) > self.dataclose(-1), 1, 0)



    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.buyprice = order.executed.price
                self.log('BUY EXECUTED, Price:%.2f, Lot:%i, Cash:%i.' %
                         (order.executed.price,
                          order.executed.size,
                          self.broker.get_cash()))

            else:
                self.sellprice = order.executed.price
                self.log('SELL EXECUTED, Price:%.2f, Lot:%i, Cash:%i.' %
                        (order.executed.price,
                          -order.executed.size,
                          self.broker.get_cash()))

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None


    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
        data_train = xgb.DMatrix(pd.DataFrame({'Bollinger':np.array(self.bollinger.get(ago=-1, size=30)), 
                                               'MACD':np.array(self.macdhisto.get(ago=-1, size=30)),
                                               'wr':np.array(self.wr.get(ago=-1, size=30))}), 
                                 label=np.array(self.updown.get(size=30)))
        data_test = xgb.DMatrix(pd.DataFrame({'Bollinger':np.array(self.bollinger.get(size=1)), 
                                              'MACD':np.array(self.macdhisto.get(size=1)), 
                                              'wr':np.array(self.wr.get(size=1))}))
        params={'booster':'gbtree',
                'nthread':12,
                'objective': 'rank:pairwise',
                'eval_metric':'auc',
                'seed':0,
                'eta': 0.01,
                'gamma':0.1,
                'min_child_weight':1.1,
                'max_depth':5,
                'lambda':10,
                'subsample':0.7,
                'colsample_bytree':0.7,
                'colsample_bylevel':0.7,
                'tree_method':'exact'
                }
        model = xgb.train(params,data_train)
        if model.predict(data_test)[0] > 0.51 and self.updown[-1] == 1 and self.updown[-2] == 1 and self.getposition(self.data).size == 0 and self.dataclose[0] > self.bollinger.mid[0]:
            cash_ava = self.broker.get_cash()
            lots = np.floor(cash_ava / (self.params.p_stake*self.dataclose[0]))*self.params.p_stake
            self.order = self.buy(size=lots)
        elif model.predict(data_test)[0] < 0.5:
            self.order = self.close()

    
    #def stop(self):
    #    self.log('Ending Position:%i. Ending Value:%.2f.' %
    #            (self.getposition(self.data).size,
    #            self.cerebro.broker.getvalue()), doprint=False)

In [64]:
if __name__ ==  '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    strats = cerebro.addstrategy(ml1, printlog=True) 

    s_date = datetime.date(2020,12,31) - datetime.timedelta(days=385)
    e_date = datetime.date(2020,12,31)

    # Create stock Data Feed
    stock_index = '000166.SZ'
    #df = utilsJ.stock_tushare('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6', 
    #                    stock_index, s_date, e_date)
    df = pd.read_csv('C:\\Users\\a\\Desktop\\Backtrader\\Data\\2019\\' + stock_index + '.csv', 
                    converters={'trade_date':lambda x:pd.to_datetime(x)}).set_index('trade_date')
    data = bt.feeds.PandasData(dataname=df,fromdate=s_date,todate=e_date)

    # Add the index Data Feed to Cerebo
    cerebro.adddata(data)

    # Set cash inside the strategy
    cerebro.broker = bt.brokers.BackBroker(coc=True)   
    cerebro.broker.setcash(20000)

    # Set commission
    #cerebro.broker.setcommission()

    # Print out the starting conditions
    start_value = cerebro.broker.getvalue()
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    final_value = cerebro.broker.getvalue()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print('Net Profit: %.2f%%' % ((final_value - start_value) / start_value * 100))

    # Visualization
    cerebro.plot(iplot=False)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\a\\Desktop\\Backtrader\\Data\\2019\\000948.SZ.csv'